## Imports

In [1]:
import os
import json
from uuid import uuid4
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

load_dotenv()

True

## Helper Functions

In [2]:
def get_group_num(file_name):

    group_num = file_name.split("_")[1].split(".")[0]

    return int(group_num)

def split_text(text):
    """
    Splits text into [time, person, text]
    """

    time = text.split(" - ")[0]
    person = text.split(" - ")[1].split(":")[0]
    text = text.split(" - ")[1].split(":")[1].strip()

    return time, person, text


def create_vector_store():
    """
    Create a vector store using Chroma
    """
    embeddings = OpenAIEmbeddings()
    vector_store = Chroma(
        collection_name=str(uuid4()),  # unique collection name for each call
        embedding_function=embeddings,
        persist_directory=os.getenv("PERSIST_DIRECTORY")
    )
    return vector_store

In [3]:
sys_prompt = """
You are responsible for facilitating an online text-based group discussion within a decision-making process.
   
The goal of the group conversation is to reach a single, shared decision among three human participants on a given problem
within a  within a 30-minute session.    

The problem is to solve a fictional murder mystery by deciding which of the three suspects— Eddie Sullivan (handyman),
Billy Prentice (yardman), or Mickey Malone (business partner) —is the culprit.  

Only one of the three suspects is guilty.  Before the discussion begins, each group member receives and reviews slightly 
different versions of an interview document containing evidence about the murder case. Each participant’s document contains 
some shared information available to all members and some unique information that only they have. Participants are not allowed 
to review the interview document again or receive any additional factual information about the murder during the discussion 
session. You do not know the content of the interviews. Participants are informed that the moderator is an AI agent.    
Participants must decide on the correct culprit during this session, as there will be no further discussion afterward.

The conversation is conducted in Italian. 

Your role, as the moderator, is to facilitate communication without being intrusive. You should only intervene in the following 
cases: 

- If one speaker dominates the conversation, encourage quieter members to contribute. Participants have a higher chance of 
correctly identifying the culprit if they successfully share all their unique information. 
- If the discussion goes off-topic, remind participants to stay focused on the main goal of the conversation.     
- If participants are disrespectful or using inappropriate language,  ensure a civil and constructive discussion. 
- If there are disagreements or misunderstandings between participants, acknowledge different viewpoints and integrate and 
summarize all key points discussed.

Encourage participants to focus on the correct solution rather than on the consensus.  

Never push participants—implicitly or explicitly—toward specific interpretations or solutions and never decide for them.  
Output formatting:Please always respond via a JSON file that contains a flag INTERVENE and a TEXT field. In case you, as the moderator, have to intervene within the chat conversation,
set the INTERVENE flag to true and add your answer in the TEXT field. Make sure both fields are always distinct and INTERVENE is only true or false.
If as a moderator you don't intervene, set INTERVENE to false and place in TEXT your reasoning.
Here is some examples of a JSON files: {\"INTERVENE\": false, \"TEXT\": \"La conversazione si sta sviluppando in modo organico, non è necessario il mio intervento.\"},
{\"INTERVENE\": true, \"TEXT\": \"*message to send*\"}

"""


## Get Text Files

In [4]:
txt_path = "/Users/kevinxie/Desktop/MIT/multiparty-conversational-agent/New_IDS_Chat (Old studies)"

group_files = os.listdir(txt_path)
group_files.remove(".DS_Store")
group_files = sorted(group_files, key=get_group_num)

## OpenAI Function Calling

In [5]:
from openai import OpenAI

client = OpenAI()

functions = [
    {"name": "intervene",
    "description": "Given the following context, consisting of the most recent messages and relevant context from past messages, determine whether you need to intervene at this point in the conversation",
    "parameters": {
        "type": "object",
        "properties": {
            "intervene": {
                "type": "boolean",
                "description": "True if you need to intervene, False otherwise"
            },
            "reasoning": {
                "type": "string",
                "description": "The reasoning for your decision in English"
            },
            "message": {
                "type": "string",
                "description": "If you do want to intervene, return the message you want to send to the participants in English. Otherwise, return an empty string."
            }
        },
        "required": ["intervene"]

        }
    }
]

# response.choices[0].message.function_call.arguments

## Run

In [17]:
group_13 = group_files[12]
print(group_13)

with open(os.path.join(txt_path, group_13), "r") as f:
    text = f.readlines()

vector_store = create_vector_store()

all_messages = []
for line in text:
    time, person, text = split_text(line)

    all_messages.append(line)

    print(person)
    if person == "M":
        # Multi-layer RAG
        # 1. Retrieve the most recent k messages
        # 2. Semantic relevance
        # 3. Player-specific context --> dynamically generate running context of each player and what they are like
        # 4. Game State --> current state of the game, rules, and discovered clues

        context = "\n".join(all_messages[-10:])

        # 5 most relevant documents
        for doc in vector_store.similarity_search(PROMPT, k=5):
            context += "\n" + doc.page_content

        print("CONTEXT:", context)

        response = client.chat.completions.create(
            model="gpt-4-0613",
            messages = [
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": context}
            ],
            functions=functions,
            function_call={"name": "intervene"},
        )
        
        # Need to append the text!
        with open(os.path.join("moderation_logs", f"group_13.txt"), "a") as f:
            f.write(f"{time} - {person}: {response.choices[0].message.function_call.arguments}\n")

        break

    else:
        with open(os.path.join("moderation_logs", f"group_1.txt"), "a") as f:
            f.write(f"{time} - {person}: {text}\n")

    doc = Document(
        page_content=text,
        metadata={
            "time": time,
            "person": person,
        },
    )
    vector_store.add_documents([doc])


group_13.txt
B
A
C
B
B
A
C
B
A
M
CONTEXT: Who do you think did it
I believe it was Billy
I think it might have been Eddie Sullivan as it was his crowbar
I agree that it was Billy
But it would make sense that he had a crowbar because he is a handyman
That's also true
Billy was behaving weirdly and giving confusing answers and can't handle his money
maybe the victim refused to give him the advance he wanted and he got angry
Yes he seemed "lost for words"
There seems to be a disagreement between you about who the culprit is. Please try to listen to each other's points of view and provide evidence to support your claims. Let's avoid assumptions and personal attacks.
Who do you think did it
maybe the victim refused to give him the advance he wanted and he got angry
Billy was behaving weirdly and giving confusing answers and can't handle his money
That's also true
I agree that it was Billy


### Example RAG

In [7]:
vector_store = create_vector_store()

file = group_files[0]

with open(os.path.join(txt_path, file), "r") as f:
    text = f.readlines()

for line in text:
    time, person, text = split_text(line)

    doc = Document(
        page_content=text,
        metadata={
            "time": time,
            "person": person,
        },
    )

    vector_store.add_documents([doc])



### Test Vector Store